In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
covid=pd.read_csv('/kaggle/input/covid19-in-india/covid_19_india.csv')

In [ ]:
covid.head(-5)

In [ ]:
eyaletler=covid['State/UnionTerritory'].unique()
eyaletler

In [ ]:
covid.info()

In [ ]:
covid.isnull().sum()

In [ ]:
covid.head(20)

In [ ]:
ss = covid.loc[(covid['State/UnionTerritory'] == 'Punjab')]

In [ ]:
ss.head(20)

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(x='ConfirmedIndianNational',data=ss)

In [ ]:
import plotly.offline as pyoff
import plotly.graph_objs as pygo

In [ ]:
Iyilesen_sayisi = pygo.Scatter(x=ss['Date'], y=ss['Cured'], name= 'Iyilesenler')
Olen_sayisi = pygo.Scatter(x=ss['Date'], y=ss['Deaths'], name= 'Vefat Edenler')
pyoff.iplot([Iyilesen_sayisi,Olen_sayisi])

In [ ]:
sem=ss[['Confirmed']]
sem = sem.values

In [ ]:
Dizi_boyutu = int(len(sem) * 0.80)
Test_sayisi = len(sem) - Dizi_boyutu
dizi, test = sem[0:Dizi_boyutu,:], sem[Dizi_boyutu:len(sem),:]
print(len(dizi), len(test))

In [ ]:
def datasetim(dataset, ileri=1):
    dataX, dataY = [], [] 
    for i in range(len(dataset)-ileri-1):
        a = dataset[i:(i+ileri), 0]
        dataX.append(a)
        dataY.append(dataset[i + ileri, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
ileri = 2
diziX, diziY = datasetim(dizi, ileri=ileri)
testX, testY = datasetim(test, ileri=ileri)

In [ ]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()

In [ ]:
model.fit(diziX,diziY)

In [ ]:
tahmin=model.predict(testX)

In [ ]:
ss = pd.DataFrame({'Gercek': testY.flatten(), 'Tahmini': tahmin.flatten()})
ss

In [ ]:
ss.plot(kind='bar',figsize=(16,16))

In [ ]:
#Yaş aralıkları

In [ ]:
yas_gruplari = pd.read_csv('/kaggle/input/covid19-in-india/AgeGroupDetails.csv')
yas_gruplari.head(10)

In [ ]:
yas_gruplari.columns

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
sns.set(style="whitegrid")
ax = sns.barplot(x="AgeGroup", y="TotalCases", data=yas_gruplari)  

In [ ]:
covid['totalIndian'] = (covid['Cured']+covid['Deaths']+covid['Confirmed'])

In [ ]:
covid.groupby('State/UnionTerritory')['totalIndian'].sum()

In [ ]:
covid['State/UnionTerritory'].value_counts()

In [ ]:
!pip install fbprophet

In [ ]:
from fbprophet import Prophet

In [ ]:
def gelecekdurumtahmini(ds, durum, gun):
    ds_durum = ds[ds['State/UnionTerritory']==durum]
    ds_model = ds_durum[['Date','Confirmed']].rename(columns={'Date': 'ds', 'Confirmed': 'y'})
    m = Prophet()
    m.fit(ds_model)
    gelecek =  m.make_future_dataframe(periods=gun)
    tahmin=m.predict(gelecek)
    figur1 = m.plot(tahmin)
    figur2 = m.plot_components(tahmin)

In [ ]:
gelecekdurumtahmini(covid,'Punjab',20)